## BigMaze

Als *state* betrachten wir eine Position, d.h. ein Tuple (x,y).
Wir legen nur die Höhe und Breite fest und lassen uns per Zufall Wände generieren.

In [67]:
import random
random.seed(1)
height = width = 3000
anteilWaende = 0.2
walls = {(x,y) for x in range(height) for y in range(width) if random.random() < anteilWaende}
print(len(walls))
start = (height//2,0)
ziel = (height//2,width-1)

1801461


### BFS - Breitensuche

In [89]:
%%time
from collections import deque
def bfs(startstate):
    ''' 
    returns: Tuple (prev, state) 
        prev: dictionary mit den Vorgängern der untersuchten Spielstellungen,            
        state: Spielstellung, die den goaltest besteht
        wenn Ziel nicht gefunden: None, None
    '''   
    frontier =  deque([startstate])
    prev = {startstate:None}
    while frontier:
        state = frontier.popleft()  
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None, None

def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]
    path.reverse()
    return path

def nextstates(state):
    '''
    returns:  Liste mit möglichen Folgestellungen zu state
    '''
    x, y = state
    tmp = []
    for xd, yd in dirs:
        x1 = x + xd
        y1 = y + yd
        if 0 <= x1 < height and 0 <= y1 < width and (x1,y1) not in walls:
            tmp.append((x1,y1))
    return tmp    
    
def goaltest(state):
    '''
    returns: True, wenn state eine Lösung ist
    '''
    return state == ziel


# Aufruf:
prev, goal = bfs(start)
print(f'{len(prev)=}')
path = reconstructPath(prev,goal) 
print(f'{len(path)=}')

len(prev)=6217435
len(path)=3430
CPU times: total: 16.2 s
Wall time: 17.8 s


### A*



In [80]:
%%time
from heapq import heappop, heappush
def astar(s):
    frontier =[(h(s),s)]  
    prev = {s:None}
    g = {s:0}                         # die Rückwärtskosten
    while frontier:
        _ ,state = heappop(frontier)  # die Kosten braucht man an der Stelle nicht
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            gg = g[state] + 1       
            if v not in prev or gg < g[v]:     
                g[v] = gg                     
                heappush(frontier,(g[v]+h(v),v))  
                prev[v] = state
    return None, None

def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]
    path.reverse()
    return path

def h(state):
    '''
    returns: Vorwärtskosten laut Heuristik
    '''
    x, y = state
    x1, y1 = ziel
    return abs(x1-x)+abs(y1-y)

#Aufruf:
prev, goal = astar(start)
print(f'{len(prev)=}')
path = reconstructPath(prev,goal) 
print(f'{len(path)=}')
 

len(prev)=719913
len(path)=3430
CPU times: total: 3.81 s
Wall time: 3.89 s


### Beamsearch

In [84]:
%%time
def beamsearch(startstate, beamwidth): 
    frontier =[startstate]
    prev = {startstate:None}
    nextfrontier = []
    while frontier:
        for state in frontier:
            if goaltest(state):
                return prev, state
            for v in nextstates(state):
                if v not in prev:
                    prev[v] = state
                    nextfrontier.append(v)
        
        frontier = sorted(nextfrontier,key=lambda x: h(x)) 
        frontier = frontier[:beamwidth]
        nextfrontier = []
    return None, None

def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]
    path.reverse()
    return path

def h(state):
    '''
    returns: Vorwärtskosten laut Heuristik
    '''
    x, y = state
    x1, y1 = ziel
    return abs(x1-x)+abs(y1-y)
    
#Aufruf:
BEAMWIDTH = 30                    # bei BEAMWIDTH=26 wird nicht mehr die optimale Lösung gefunden
prev, goal = beamsearch(start,BEAMWIDTH)
print(f'{len(prev)=}')
path = reconstructPath(prev,goal) 
print(f'{len(path)=}')


len(prev)=102918
len(path)=3430
CPU times: total: 438 ms
Wall time: 518 ms


In [1]:
def getMove(s1, s2):
    '''
    returns: die Beschreibung des Übergangs von state s1 zu state s2
    '''
    pass

def getMoves(path):
    '''
    returns: Beschreibung des Pfads als eine Folge von Aktionen (Moves)
    '''
    moves = ''
    for i in range(len(path)-1):
        moves+=getMove(path[i],path[i+1])
    return moves

#Aufruf
#print(getMoves(path))